In this Notebook we will explore different combinations of input parameters (lowfreq, upfreq, maxdist, and dcutoff) in order to find the optimal combination to compute reliable 3D models of our regions of interest \*. The Notebook will generate files with the correlation between the model ensembles and the input interaction matrix data that will be assessed in 02_chooseBestParameters.ipynb in order to select the best parameters combination, or, if needed, define a different range of input parameters to continue the exploration.
 
 \* Serra, F., Baù, D., Goodstadt, M., Castillo, D. Filion, G., & Marti-Renom, M.A. (2017). Automatic analysis and 3D-modelling of Hi-C data using TADbit reveals structural features of the fly chromatin colors. PLOS Comp Bio 13(7) e1005665. doi:10.1371/journal.pcbi.1005665 
 
 \* Marco Di Stefano, Ralph Stadhouders, Irene Farabella, David Castillo, François Serra, Thomas Graf, Marc A. Marti-Renom. Transcriptional activation during cell reprogramming correlates with the formation of 3D open chromatin hubs. Nature Communications, 11(1). ISSN 20411723.; doi: 10.1038/s41467-020-16396-1

# Libraries and functions 

In [1]:
from taddyn import Chromosome, Experiment
import numpy as np
import os
import shutil
from taddyn.modelling.impoptimizer  import IMPoptimizer
import copy
import sys
import datetime

# Parameters to modify

Now we will define a range of  parameter combinations in which we will check how well the models represent the input data.  

NOTE: The interaction matrix should ideally be included inside a structure of folders without '\_'. Nested inside of a folder that differentiates first the cell type and then the project or regions. Then it must start by Matrix, and contain the following elements separated by '\_':  
.../Cell/Project/[Matrix]_[Cell]_[region]_[Chromosome]-[startCoord]-[EndCoord]_[resolution]bp  

Example:  
.../Ery/b-globin/MatrixNormFreqFiltrd_Ery_b-globin_chr11-4615000-6175000_5000bp
In this example, all matrices will be in a tree of folders inside matrices

You can find a guide to decide the range of parameters to check for your regions of interest in here:  
https://github.com/3DGenomes/MethodsMolBiol/blob/master/Notebooks/Methods-10-Modeling%20parameters%20optimization.ipynb  

The following code explores lowfreq values ranging from -1 to 0.5 in steps of 0.5, upfreq values from -0.5 to 0.5 in steps of 0.5, and maxdist range from 200 to 500 in steps of 100.
Note, the code allowed for the exploration of the dcutoff parameters in a range starting from 100 (two times resolution_scale) to 300 in steps of 100. 

In [3]:
## Optimisation parameters
# range of lowfreq values
lowfreq_arange = np.arange(-1.0,1.0,0.5)
# range of dcutoff values
dcutoff_range= [100,400,100]  # start, end + step, step 
# range of maxdist values
m_range= np.arange(200,500,100)
# upfreq range values
upfreq_range= np.arange(-0.5,1.0,0.5)
# number of CPU to use
cpus = 8 
# number of models to reconstruct in the optimisation
nmodels = 100
# days-hours:minutes:seconds of maximum running time per job
jobTime = '0-08:00:00' 

## Data Paths (Location of the base folder downloaded from GitHub)
basePath = '/home/julen/TADdyn/TADdyn_tutorial/'

# Run

## Define additional paths 

In [4]:
scriptsPath = basePath + 'code/modellingScripts/'
tempOut = basePath + 'temporal/'

## Import additional libraries  

In [5]:
sys.path.append(basePath + 'code')
import fileHandling
import metrics

## Get matrix paths 

In [6]:
matricesLength, regionsAll, matrices = fileHandling.getMatricesPaths(basePath, starting='Matrix')

## Preparing optimisation run

This code will create a set of commands to run the optimisation the given combination of parameters

### Get combinations of paramteres and run commands 

In [34]:
combinations = {}
for cell in matrices:
    combinations[cell] = {}
    for regi in matrices[cell]:
        matPath = matrices[cell][regi]
        c_rangeDot= metrics.dotToC('_'.join(str(j) for j in dcutoff_range))
        combinations[cell][regi] = metrics.getParamCombi(lowfreq_arange, m_range, c_rangeDot, upfreq_range, 
                         scriptsPath, dcutoff_range, matPath, jobTime, nmodels,
                         tempOut, cpu=cpus)

### Stimate total modelling time

It is important to ensure that the number of models you want to build in the optimisation will more or less be resolved in an acceptable time. The modelling timing is highly dependent on both the number of particles of the models (which depends on the length of the region and the resolution of the experiment) and the number of CPU you provide for the modelling process. Please, pay special attention to the output of the following sections and adjust the region to be modelled, its resolution, or the number of CPUs accordingly.

WARNING: We discorage users to work with models bigger than 1000 bins/particles due to the exponential increase in modelling timing.

In [8]:
for regi in regionsAll:
    print('--- %s ---' %regi)
    ncell = len(matrices.keys())
    print('Counting %s cells' %ncell)
    ncombi = len(combinations[cell][regi]) * ncell
    
    # each combination has n models, so we need to multiply
    totalModels = ncombi * nmodels
    
    # Finally we get the total time to do all the models in each region
    timePerModel = metrics.stimateTime(matricesLength[regi])
    totalTime = totalModels * timePerModel
    totalTime2 = str(datetime.timedelta(seconds=totalTime))
    
    print('%s models will be computed, in a median stimated time (with 1 CPU) of %s' %(
                                        totalModels, totalTime2))
    
    totalTime2 = str(datetime.timedelta(seconds=totalTime/cpus))
    print("Stimated time with assigned number of %s CPU's: %s" %(cpus,
                                        totalTime2))
    print('')

--- b-globin ---
Counting 3 cells
9000 models will be computed, in a median stimated time (with 1 CPU) of 84 days, 4:46:30
Stimated time with assigned number of 8 CPU's: 10 days, 12:35:48.750000



### Run optimisation 

Output files with the correlation of the models from each of the combinations will be stored in the same folder as each of the matrix.  

Example: opt_LF-1.0UF0.0C100.0-200.0-300.0Mdis200_5000bp.txt  

In the same folder, a folder called lammpsSteps will be created to store the models of the combinations that yet need to be started or finished without computing the asked number of models.

In [18]:
for cell in combinations:
    print('## %s ##' %(cell))
    for regi in combinations[cell]:
        print('--- %s ---' %regi)
        for nc, combi in enumerate(combinations[cell][regi]):
            print('Combination %s' %(nc))
            ! python {combi}

## Ery ##
--- b-globin ---
Combination 0
####
Experiment test:
   resolution        : 5 kb
   TADs              : None
   Hi-C rows         : 942
   normalized        : visibility
   identifier        : GM128Rao
   cell type         : UNKNOWN
   restriction enzyme: UNKNOWN

942
Optimizing 80 particles
  num scale	kbending	maxdist	lowfreq	upfreq	dcutoff	correlation
Performing minimization run...
Performing minimization run...
Performing minimization run...
Performing minimization run...
Performing minimization run...
Performing minimization run...
Performing minimization run...
Performing minimization run...
Performing minimization run...
Performing minimization run...
  1   0.01 	0       	200    	-1     	-1    	200    0.4514
  2   0.01 	0       	200    	-1     	-1    	100    0.7238
All models finished correctly
Combination 1
####
Experiment test:
   resolution        : 5 kb
   TADs              : None
   Hi-C rows         : 942
   normalized        : visibility
   identifier        : G

### Continuing optimisation run

NOTE: Some times the run will not reach the end, in these cases we restart them starting from a different initial random positioning of the particles. By default this step will be repeated at most 10 times.

In [36]:
## we will rerun the models until we finish all them or we reach 
# 10 steps
combinations_t = copy.deepcopy(combinations)
for cell in combinations_t:
    print('## %s ##' %(cell))
    for regi in combinations_t[cell]:
        print('--- %s ---' %regi)
        matPath = matrices[cell][regi]
        nchecks = 0
        while len(combinations_t[cell][regi]) > 0 or nchecks < 10:
            combinations2 = copy.copy(combinations_t[cell][regi])
            combinations_t[cell][regi] = []
            for nc, combi in enumerate(combinations2):
                # get paths and check if the modelling finished
                path = '/'.join(matPath.split('/')[:-1]) + '/'
                jobName = 'LF%sUF%sMdis%s_%sbp' %(combi.split()[2], combi.split()[8], combi.split()[6], 
                                            matPath.split('_')[-1][:-2])
                keep_restart_out_dir = path + 'lammpsSteps/jobArray_%s/' %jobName
                if os.path.isdir(keep_restart_out_dir):
                    print('Combination %s' %(nc))
                    # then it didnt finished
                    combinations_t += [combi]
                    ! python {combi}
            nchecks += 1
        
        

## Mon ##
--- b-globin ---
## Ery ##
--- b-globin ---
## nCD4 ##
--- b-globin ---


## clean temporal folders 

The content inside of the generated temporal folders inside $tempOut is usually deleted after the modelling successfully finishes, not the folders itself though. Besides, if the modelling process breaks for any reason, the temporal files will remain there. TADdyn modelling generates a lot of temporal files so, in order to ensure that they don't accumulate, we will remove the container folders after each optimisation process.

In [8]:
tempFolders = os.listdir(tempOut)

In [10]:
for t in tempFolders:
    print(t)
    shutil.rmtree(tempOut + t)

sqzn
uyxb
CRil
cVeT
GAZl
xTjj
GbFD
epME
UGbc
GrNS
